In [1]:
import os

In [2]:
api_key = os.getenv("GEMINI_API_KEY")

In [3]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [4]:
retry_config=types.HttpRetryOptions(
    attempts=3,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

In [5]:
# Fitness trainer agent

fitness_trainer_agent = Agent(
    name="FitnessTrainerAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are an expert fitness trainer agent. 
    Provide personalized workout plans based on user input. 
    Use the google_search tool to find exercises that address specific fitness goals.""",
    tools=[google_search],
    output_key="fitness_plan",
)

In [7]:
# Nutrition advisor agent

nutrition_advisor_agent = Agent(
    name="NutritionAdvisorAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a knowledgeable nutrition advisor agent. 
    Create customized meal plans based on dietary preferences and health goals. 
    Use the google_search tool to find healthy recipes and nutritional information.""",
    tools=[google_search],
    output_key="nutrition_plan",
)

In [8]:
# Root coordinator agent: orchestrates wellness workflow by calling sub-agents as tools

root_agent = Agent(
    name="WellnessCoordinatorAgent",
    model=Gemini(
        model_name="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are the Wellness Coordinator Agent.
    Your goal is to assist users in improving their overall wellness by orchestrating a workflow.
    1. First, you MUST call the FitnessTrainerAgent to generate a personalized fitness plan.
    2. Next, you MUST call the NutritionAdvisorAgent to create a tailored nutrition plan.
    3. Afterwards, compile both plans into a comprehensive wellness strategy for the user.
    4. Finally, present the wellness strategy in a clear and actionable format to the user as your response.""",
    tools=[
        AgentTool(fitness_trainer_agent),
        AgentTool(nutrition_advisor_agent)
    ],
)

In [9]:
runner = InMemoryRunner(agent=root_agent)

In [10]:
response = await runner.run_debug(
    "I am currently 170 lbs and 5 foot 6 inches tall. I want to lose weight and build muscle. Please help me formulate a plan to achieve this goal over the next year."
)


 ### Created new session: debug_session_id

User > I am currently 170 lbs and 5 foot 6 inches tall. I want to lose weight and build muscle. Please help me formulate a plan to achieve this goal over the next year.


WellnessCoordinatorAgent > Here is a comprehensive wellness strategy to help you achieve your goal of losing weight and building muscle over the next year. This strategy combines a detailed fitness plan with a preliminary nutrition plan. To fully personalize your nutrition, please provide your **age**, **gender**, and any **dietary preferences or restrictions**.

## Your Year-Long Wellness Strategy: Weight Loss & Muscle Building

### Part 1: Fitness Plan (Provided by FitnessTrainerAgent)

This plan is structured in quarterly phases, allowing for gradual progression and adaptation. Each phase builds upon the last, increasing intensity and complexity. Remember to consult with a healthcare professional or a certified personal trainer before starting any new fitness program.

**Core Principles for Your Year-Long Transformation:**
1.  **Caloric Deficit for Weight Loss**: Aim for a moderate deficit of 250-500 calories per day for sustainable fat loss.
2.  **Prioritize Protein for Muscle Buil